# target encoder & ohe & DeepFM - predict

In [1]:
%load_ext autoreload
%autoreload 2

import os
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append(os.path.abspath('..'))
# ---------------------------------
from time import sleep
import numpy as np
import pandas as pd
import scipy
import tqdm
import tensorflow as tf
from tensorflow.keras.layers import Activation
import matplotlib.pyplot as plt

from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression, SGDRegressor
from sklearn.metrics import roc_auc_score

from hyperopt import hp
# ---------------------------------
from tools import CV, Tuning, CVGetScore, IdxValEncoder, deepfm, CyclicLR, MaxLrFinder
# ---------------------------------
from tools import focal_loss, gelu, mish
from tensorflow.keras.utils import get_custom_objects

get_custom_objects().update({'focal_loss': focal_loss()})
get_custom_objects().update({'mish': mish})
get_custom_objects().update({'gelu': gelu})

In [2]:
train_df = pd.read_csv('../data/train.csv', index_col='id')
test_df = pd.read_csv('../data/test.csv', index_col='id')

# ord_5
for i in range(2):
    train_df[f'ord_5_{i}'] = train_df['ord_5'].str[i]
    test_df[f'ord_5_{i}'] = test_df['ord_5'].str[i]

# null
train_df['null'] = train_df.isna().sum(axis=1)
test_df['null'] = test_df.isna().sum(axis=1)

for col in test_df.columns:
    train_df[col].fillna('isnull', inplace=True)
    test_df[col].fillna('isnull', inplace=True)

# target
target = train_df['target']
y_train = target.values

# drop
train_df.drop(['target', 'ord_5'], axis=1, inplace=True)
test_df.drop(['ord_5'], axis=1, inplace=True)

In [3]:
feature_col = train_df.columns

bin_col = ['null']

class_col = ['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4',
             'nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4',
             'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9',
             'ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4',
             'day', 'month', 'ord_5_0', 'ord_5_1']

In [4]:
# for col in bin_col:
#     map_dict = dict(zip(train_df[col].unique(), [0., 1.]))
#     train_df[col] = train_df[col].map(map_dict)
#     test_df[col] = test_df[col].map(map_dict)

In [5]:
ecd = IdxValEncoder(feature_col, bin_col=bin_col, class_col=class_col)
ecd.fit(train_df, verbose=1)

600000it [00:11, 50923.28it/s]


In [6]:
ecd.fit(test_df, verbose=1)

400000it [00:07, 55339.47it/s]


In [7]:
idx, val = ecd.transform(train_df, verbose=1)
idx_test, val_test = ecd.transform(test_df, verbose=1)

600000it [00:13, 45507.27it/s]
400000it [00:08, 46938.62it/s]


# Tuning

In [27]:
%matplotlib qt5

batch_size = 8192
epochs = 100
nflod = 5

# fit param

clr = CyclicLR(base_lr=1e-5,
               max_lr = 1e-3, 
               step_size= int(4.0*(train_df.shape[0]*((nflod-1)/nflod)) / batch_size),
               mode='exp_range',
               gamma=1.0)

es = tf.keras.callbacks.EarlyStopping(monitor='val_AUC', 
                                      patience=3,
                                      mode='max',
                                      restore_best_weights=True)

fit_param = {
    'batch_size': batch_size, 
    'epochs':epochs, 
    'verbose': 0,
    'callbacks':[es, clr]
}

cv_fit_param = {'fit_params': fit_param, 
                'eval_param': {'batch_size':batch_size},
                'use_proba':False, 
                'fit_use_valid': True}

# model_fix_param & model_search_space
model_fix_param = {'vocabulary_size':ecd.get_vocabulary(), 
                   'feature_number': len(feature_col),
                   'activation': 'sigmoid',
                   'metrics': ['AUC'],
                   'use_fm': True,
                   'k': 5,
                   'deep_use_bn': False,
                   'optimizer': 'Adam',
                   'loss': 'binary_crossentropy',
                   'num_deep_layer':2}

ss = {
    'deep_activation': (hp.choice, ('mish', 'gelu')),
    'num_neuron': (hp.choice, (64, 128, 256)),
    'l2_deep': (hp.loguniform, (-15, -6)),
    'l2_pair': (hp.loguniform, (-10, -5)),
    'deep_dropout': (hp.uniform, (0, 0.5)),
}

# cv get score
def neg_auc(y_true, y_pred):
    return - roc_auc_score(y_true, y_pred)

gs = CVGetScore(x=[idx, val],
                y=y_train, 
                metrics_func=neg_auc,
                split_method=StratifiedKFold,
                nfolds=nflod, 
                random_state=2333,
                model=deepfm, 
                cv_fit_params=cv_fit_param, 
                model_fix_params=model_fix_param, 
                model_search_space=ss)

tuning = Tuning(gs, verbose=1)
tuning.fmin(gs.GET_SEARCH_SPACE(), max_evals=100)

new best, eval 1, score -0.7885, param {'deep_activation': 'mish', 'deep_dropout': 0.00981216322002143, 'l2_deep': 4.9180264771620935e-05, 'l2_pair': 0.004222828750261717, 'num_neuron': 128}
new best, eval 7, score -0.7885, param {'deep_activation': 'mish', 'deep_dropout': 0.19703628657830424, 'l2_deep': 6.340268164675297e-06, 'l2_pair': 0.003764648850841062, 'num_neuron': 256}
new best, eval 9, score -0.7886, param {'deep_activation': 'mish', 'deep_dropout': 0.12538472404066425, 'l2_deep': 5.503630152514677e-05, 'l2_pair': 0.0016918737987316752, 'num_neuron': 128}
new best, eval 14, score -0.7886, param {'deep_activation': 'gelu', 'deep_dropout': 0.4078779650580133, 'l2_deep': 0.00010670103072757633, 'l2_pair': 0.0012082082974794858, 'num_neuron': 256}
new best, eval 23, score -0.7887, param {'deep_activation': 'gelu', 'deep_dropout': 0.10158096988433507, 'l2_deep': 0.00010445003470729774, 'l2_pair': 0.0009393900912083867, 'num_neuron': 128}
new best, eval 38, score -0.7887, param {'d

{'deep_activation': 1,
 'deep_dropout': 0.036687501496120156,
 'l2_deep': 3.922971773395442e-05,
 'l2_pair': 0.0019189114102563794,
 'num_neuron': 2}

In [28]:
log = tuning.log.get_log()

In [29]:
log.sort_values('score').head(20)

,score,update,usetime,deep_activation,deep_dropout,l2_deep,l2_pair,num_neuron
37,-0.788738,True,184.576666,gelu,0.036688,0.000039,0.001919,256
22,-0.788713,True,161.217921,gelu,0.101581,0.000104,0.000939,128
59,-0.788669,False,135.140283,gelu,0.091591,0.000008,0.003207,128
40,-0.788648,False,130.189999,gelu,0.044670,0.000012,0.001696,128
53,-0.788645,False,180.608015,gelu,0.155234,0.000110,0.000710,256
68,-0.788643,False,171.826066,gelu,0.128133,0.000045,0.001630,128
13,-0.788623,True,221.660247,gelu,0.407878,0.000107,0.001208,256
85,-0.788617,False,194.853450,gelu,0.367318,0.000006,0.002529,256
55,-0.788603,False,214.681732,gelu,0.118396,0.000085,0.001304,256
96,-0.788595,False,224.605684,gelu,0.176778,0.000157,0.000690,256


In [30]:
log.sort_values('score').tail(20)

,score,update,usetime,deep_activation,deep_dropout,l2_deep,l2_pair,num_neuron
93,-0.787830,False,364.312644,gelu,0.423106,0.000696,0.006660,64
50,-0.787820,False,281.489749,gelu,0.190125,0.000161,0.004162,128
23,-0.787806,False,596.673617,gelu,0.300138,0.002016,0.001052,256
52,-0.787801,False,241.253534,gelu,0.006285,0.001265,0.000185,128
31,-0.787789,False,304.279882,gelu,0.057843,0.001574,0.000148,128
19,-0.787719,False,90.217889,gelu,0.179253,0.000005,0.000051,64
46,-0.787694,False,427.626535,gelu,0.073718,0.000382,0.001842,256
87,-0.787625,False,249.450717,gelu,0.188108,0.000457,0.000958,64
77,-0.787618,False,261.287885,gelu,0.095602,0.000194,0.004049,64
58,-0.787607,False,162.225496,mish,0.017023,0.002391,0.000061,256


In [15]:
log.to_csv('../tmp/deepfm/03051921.csv', index=False)

# pred

In [8]:
log = pd.read_csv('../tmp/deepfm/03051921.csv')

In [14]:
log

,score,update,usetime,deep_activation,deep_dropout,l2_deep,l2_pair,num_neuron
0,-0.788458,True,138.033633,mish,0.009812,0.000049,0.004223,128
1,-0.788146,False,164.677754,gelu,0.441774,0.000005,0.000633,256
2,-0.788361,False,147.270837,mish,0.410327,0.000663,0.000093,256
3,-0.787570,False,252.504611,mish,0.328100,0.001100,0.000690,64
4,-0.788395,False,156.501965,gelu,0.377664,0.000259,0.000087,256
...,...,...,...,...,...,...,...,...
95,-0.787368,False,288.332807,gelu,0.095589,0.000207,0.003818,128
96,-0.788595,False,224.605684,gelu,0.176778,0.000157,0.000690,256
97,-0.788138,False,118.662163,mish,0.038866,0.000007,0.004707,128
98,-0.788140,False,138.699083,gelu,0.108577,0.000005,0.002791,128


In [16]:
batch_size = 8192
epochs = 100
nflod = 20
nmodel = 10

In [17]:
# model params
model_tuning_param = log.sort_values('score').head(nmodel).reset_index(drop=True).to_dict()

model_fix_param = {'vocabulary_size':ecd.get_vocabulary(), 
                   'feature_number': len(feature_col),
                   'activation': 'sigmoid',
                   'metrics': ['AUC'],
                   'use_fm': True,
                   'k': 5,
                   'deep_use_bn': False,
                   'optimizer': 'Adam',
                   'loss': 'binary_crossentropy',
                   'num_deep_layer':2}

In [18]:
# callbacks
clr = CyclicLR(
    base_lr=1e-5,
    max_lr = 1e-3, 
    step_size= int(4.0*(train_df.shape[0]*((nflod-1)/nflod)) / batch_size),
    mode='exp_range',
    gamma=1.0)

es = tf.keras.callbacks.EarlyStopping(monitor='val_AUC', 
                                      patience=3,
                                      mode='max',
                                      restore_best_weights=True)

# fit
fit_param = {
    'batch_size': batch_size, 
    'epochs':epochs, 
    'verbose': 1,
    'callbacks':[es, clr]
}

cv_fit_param = {
    'fit_params': fit_param, 
    'eval_param': {'batch_size':batch_size},
    'use_proba':False, 
    'fit_use_valid': True
}

In [19]:
pred_lst = []
score_lst = []

for i in range(nmodel):
    model_params = {}
    for param_name, param_value in model_fix_param.items():
        model_params[param_name] = param_value
        
    for param_name in model_tuning_param.keys():
        if param_name not in ['score', 'update', 'usetime', 'index']:
            model_params[param_name] = model_tuning_param[param_name][i]
            
    # cv
    model = deepfm(**model_params)
    cv = CV(model, nflod)
    
    score = cv.fit(x=[idx, val],
                   y=y_train, 
                   metrics_func=roc_auc_score,
                   split_method=StratifiedKFold,
                   fit_params=fit_param,
                   eval_param={'batch_size':batch_size},
                   use_proba=False, 
                   verbose=True,
                   fit_use_valid=True)
    
    pred = cv.predict(x=[idx_test, val_test], pred_param={'batch_size': batch_size})
    pred_lst.append(pred)
    score_lst.append(score)

    tf.keras.backend.clear_session()

Train on 570000 samples, validate on 30000 samples
Epoch 1/100
570000/570000 [==============================] - 7s 13us/sample - loss: 0.7491 - AUC: 0.4869 - val_loss: 0.7401 - val_AUC: 0.4937
Epoch 2/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.7222 - AUC: 0.5069 - val_loss: 0.6938 - val_AUC: 0.5320
Epoch 3/100
570000/570000 [==============================] - 5s 10us/sample - loss: 0.6328 - AUC: 0.5131 - val_loss: 0.5549 - val_AUC: 0.5588
Epoch 4/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.5229 - AUC: 0.5581 - val_loss: 0.5092 - val_AUC: 0.6275
Epoch 5/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.5007 - AUC: 0.6670 - val_loss: 0.4934 - val_AUC: 0.6949
Epoch 6/100
570000/570000 [==============================] - 5s 10us/sample - loss: 0.4862 - AUC: 0.7162 - val_loss: 0.4812 - val_AUC: 0.7261
Epoch 7/100
570000/570000 [==============================] - 5s 10us/sample - loss: 0.4755 - AUC: 0.

Epoch 3/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.5063 - AUC: 0.6141 - val_loss: 0.4996 - val_AUC: 0.6500
Epoch 4/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4914 - AUC: 0.6847 - val_loss: 0.4810 - val_AUC: 0.7210
Epoch 5/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4682 - AUC: 0.7398 - val_loss: 0.4519 - val_AUC: 0.7592
Epoch 6/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4390 - AUC: 0.7652 - val_loss: 0.4274 - val_AUC: 0.7768
Epoch 7/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4246 - AUC: 0.7793 - val_loss: 0.4216 - val_AUC: 0.7833
Epoch 8/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4206 - AUC: 0.7844 - val_loss: 0.4195 - val_AUC: 0.7860
Epoch 9/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4187 - AUC: 0.7870 - val_loss: 0.4186 - val_AUC: 0.7871
Epoch 10/100


570000/570000 [==============================] - 5s 9us/sample - loss: 0.4094 - AUC: 0.7970 - val_loss: 0.4162 - val_AUC: 0.7860
Epoch 18/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4091 - AUC: 0.7972 - val_loss: 0.4159 - val_AUC: 0.7860
Epoch 19/100
570000/570000 [==============================] - 5s 10us/sample - loss: 0.4086 - AUC: 0.7972 - val_loss: 0.4155 - val_AUC: 0.7859
Epoch 20/100
570000/570000 [==============================] - 5s 10us/sample - loss: 0.4079 - AUC: 0.7974 - val_loss: 0.4149 - val_AUC: 0.7860
Epoch 21/100
570000/570000 [==============================] - 5s 10us/sample - loss: 0.4071 - AUC: 0.7978 - val_loss: 0.4147 - val_AUC: 0.7857
folds 4 is done, score is 0.7860668422640411
Train on 570000 samples, validate on 30000 samples
Epoch 1/100
570000/570000 [==============================] - 7s 12us/sample - loss: 0.6105 - AUC: 0.5083 - val_loss: 0.5109 - val_AUC: 0.5817
Epoch 2/100
570000/570000 [==============================] - 5

570000/570000 [==============================] - 5s 9us/sample - loss: 0.4088 - AUC: 0.7972 - val_loss: 0.4152 - val_AUC: 0.7881
Epoch 16/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4086 - AUC: 0.7972 - val_loss: 0.4149 - val_AUC: 0.7881
Epoch 17/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4082 - AUC: 0.7972 - val_loss: 0.4145 - val_AUC: 0.7880
folds 7 is done, score is 0.7880684992751308
Train on 570000 samples, validate on 30000 samples
Epoch 1/100
570000/570000 [==============================] - 7s 12us/sample - loss: 0.5688 - AUC: 0.5346 - val_loss: 0.4845 - val_AUC: 0.7045
Epoch 2/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4636 - AUC: 0.7405 - val_loss: 0.4431 - val_AUC: 0.7642
Epoch 3/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4329 - AUC: 0.7698 - val_loss: 0.4258 - val_AUC: 0.7767
Epoch 4/100
570000/570000 [==============================] - 5s 9us

Epoch 17/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4097 - AUC: 0.7969 - val_loss: 0.4153 - val_AUC: 0.7880
Epoch 18/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4094 - AUC: 0.7971 - val_loss: 0.4150 - val_AUC: 0.7880
Epoch 19/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4090 - AUC: 0.7971 - val_loss: 0.4146 - val_AUC: 0.7879
Epoch 20/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4083 - AUC: 0.7972 - val_loss: 0.4141 - val_AUC: 0.7879
folds 10 is done, score is 0.7880282263228047
Train on 570000 samples, validate on 30000 samples
Epoch 1/100
570000/570000 [==============================] - 7s 13us/sample - loss: 0.5741 - AUC: 0.5268 - val_loss: 0.4942 - val_AUC: 0.6647
Epoch 2/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4805 - AUC: 0.7127 - val_loss: 0.4697 - val_AUC: 0.7387
Epoch 3/100
570000/570000 [=========================

...

570000/570000 [==============================] - 5s 9us/sample - loss: 0.4072 - AUC: 0.7973 - val_loss: 0.4086 - val_AUC: 0.7951
Epoch 30/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4069 - AUC: 0.7974 - val_loss: 0.4085 - val_AUC: 0.7951
Epoch 31/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4068 - AUC: 0.7975 - val_loss: 0.4083 - val_AUC: 0.7951
Epoch 32/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4066 - AUC: 0.7974 - val_loss: 0.4080 - val_AUC: 0.7951
Epoch 33/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4063 - AUC: 0.7973 - val_loss: 0.4076 - val_AUC: 0.7951
Epoch 34/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4059 - AUC: 0.7971 - val_loss: 0.4070 - val_AUC: 0.7951
Epoch 35/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4053 - AUC: 0.7974 - val_loss: 0.4066 - val_AUC: 0.7951
folds 9 is done, sc

Epoch 10/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4353 - AUC: 0.7877 - val_loss: 0.4388 - val_AUC: 0.7815
Epoch 11/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4338 - AUC: 0.7888 - val_loss: 0.4379 - val_AUC: 0.7818
Epoch 12/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4324 - AUC: 0.7896 - val_loss: 0.4361 - val_AUC: 0.7827
Epoch 13/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4300 - AUC: 0.7908 - val_loss: 0.4335 - val_AUC: 0.7837
Epoch 14/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4269 - AUC: 0.7921 - val_loss: 0.4307 - val_AUC: 0.7847
Epoch 15/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4235 - AUC: 0.7936 - val_loss: 0.4280 - val_AUC: 0.7851
Epoch 16/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4208 - AUC: 0.7946 - val_loss: 0.4263 - val_AUC: 0.7854
Epoch 

570000/570000 [==============================] - 5s 9us/sample - loss: 0.4285 - AUC: 0.7933 - val_loss: 0.4302 - val_AUC: 0.7911
Epoch 18/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4279 - AUC: 0.7935 - val_loss: 0.4296 - val_AUC: 0.7912
Epoch 19/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4269 - AUC: 0.7939 - val_loss: 0.4284 - val_AUC: 0.7913
Epoch 20/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4254 - AUC: 0.7942 - val_loss: 0.4267 - val_AUC: 0.7916
Epoch 21/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4233 - AUC: 0.7948 - val_loss: 0.4248 - val_AUC: 0.7917
Epoch 22/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4212 - AUC: 0.7953 - val_loss: 0.4231 - val_AUC: 0.7919
Epoch 23/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4195 - AUC: 0.7959 - val_loss: 0.4220 - val_AUC: 0.7918
Epoch 24/100
570000

570000/570000 [==============================] - 5s 9us/sample - loss: 0.4450 - AUC: 0.7806 - val_loss: 0.4443 - val_AUC: 0.7813
Epoch 11/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4434 - AUC: 0.7818 - val_loss: 0.4422 - val_AUC: 0.7826
Epoch 12/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4405 - AUC: 0.7837 - val_loss: 0.4390 - val_AUC: 0.7845
Epoch 13/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4366 - AUC: 0.7863 - val_loss: 0.4351 - val_AUC: 0.7865
Epoch 14/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4323 - AUC: 0.7890 - val_loss: 0.4313 - val_AUC: 0.7883
Epoch 15/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4285 - AUC: 0.7914 - val_loss: 0.4289 - val_AUC: 0.7890
Epoch 16/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4260 - AUC: 0.7928 - val_loss: 0.4275 - val_AUC: 0.7895
Epoch 17/100
570000

570000/570000 [==============================] - 5s 9us/sample - loss: 0.4196 - AUC: 0.7954 - val_loss: 0.4258 - val_AUC: 0.7843
Epoch 16/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4189 - AUC: 0.7955 - val_loss: 0.4249 - val_AUC: 0.7843
Epoch 17/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4177 - AUC: 0.7957 - val_loss: 0.4236 - val_AUC: 0.7845
Epoch 18/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4161 - AUC: 0.7960 - val_loss: 0.4220 - val_AUC: 0.7845
Epoch 19/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4143 - AUC: 0.7964 - val_loss: 0.4206 - val_AUC: 0.7847
Epoch 20/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4129 - AUC: 0.7968 - val_loss: 0.4197 - val_AUC: 0.7847
Epoch 21/100
570000/570000 [==============================] - 5s 9us/sample - loss: 0.4120 - AUC: 0.7970 - val_loss: 0.4191 - val_AUC: 0.7847
Epoch 22/100
570000

In [24]:
submission = pd.read_csv('../data/sample_submission.csv', index_col='id')
submission['target'] = np.mean(pred_lst, axis=0)
submission.to_csv('../tmp/submission/main_3_deepfm030601.csv')